In [34]:
import os
import numpy as np
import cv2

In [65]:
def rearrange_channels(data):
    B, G1, G2, R = data[..., 0], data[..., 1], data[..., 2], data[..., 3]
    return np.stack([B, G1, G2, R], axis=-1)

In [88]:
def process_npz_files(input_dir, output_dir):
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.npz'):
                relative_path = os.path.relpath(root, input_dir)
                output_folder_path = os.path.join(output_dir, relative_path)
                os.makedirs(output_folder_path, exist_ok=True)

                npz_file = os.path.join(root, file)
                npz_data = np.load(npz_file)

                for key in ['sht', 'mid', 'lng', 'hdr']:
                    if key in npz_data:
                        data = npz_data[key][:, :, :4]  # Process only the original raw data
                        int_data = (data * 65535).astype(np.uint16)  # Convert float data to 16-bit

                        # Merge four channels into one following the Bayer pattern (top-left is B)
                        merged_data = np.empty((int_data.shape[0]*2, int_data.shape[1]*2), dtype=np.uint16)
                        merged_data[0::2, 0::2] = int_data[..., 0]  # B
                        merged_data[0::2, 1::2] = int_data[..., 1]  # G1
                        merged_data[1::2, 0::2] = int_data[..., 2]  # G2
                        merged_data[1::2, 1::2] = int_data[..., 3]  # R

                        demosaic_data = cv2.cvtColor(merged_data, cv2.COLOR_BayerBG2BGR) / 65535  # Demosaic and normalize
                        output_file = os.path.join(output_folder_path, f"{file[:-4]}_{key}.png")
                        cv2.imwrite(output_file, demosaic_data * 65535)


In [89]:
input_data_dir = "data/NPZ_data"
output_data_dir = "data/PNG_data"

In [ ]:
process_npz_files(input_data_dir, output_data_dir)